In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/SW')
from __future__ import print_function
from future.builtins import zip
from future.utils import iteritems
import random
from collections import Counter
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import scipy.sparse
import gensim
from nltk.corpus import stopwords
from multiprocessing import Pool
import pickle
import os
import nltk
import numpy as np
import gensim
from gensim import utils
import numpy as np
import pickle

from scipy import sparse
from scipy.stats import dirichlet
from scipy.stats import multinomial
from scipy.stats import zipf
from scipy.spatial.distance import cdist
from scipy.linalg import norm
from pyartm import regularizers
from pyartm.optimizations import gradient
from pyartm.optimizations import default
from pyartm.common import experiments
from pyartm_experiments.common import default_plot  
import pyartm_datasets
from pyartm_datasets import main_cases
from pyartm_datasets import sklearn_dataset
from pyartm_datasets import nips
from pyartm_datasets import twitter_sentiment140
from sklearn.datasets import fetch_20newsgroups
from pyartm.optimizations import balanced
%matplotlib inline

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/SW')
from __future__ import print_function
from future.builtins import zip
from future.utils import iteritems
import random
from collections import Counter
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import scipy.sparse
import gensim
from nltk.corpus import stopwords
from multiprocessing import Pool
import pickle
import os
import nltk
import numpy as np
import gensim
from gensim import utils
import numpy as np
import pickle

from scipy import sparse
from scipy.stats import dirichlet
from scipy.stats import multinomial
from scipy.stats import zipf
from scipy.spatial.distance import cdist
from scipy.linalg import norm
from pyartm import regularizers
from pyartm.optimizations import gradient
from pyartm.optimizations import default
from pyartm.common import experiments
from pyartm_experiments.common import default_plot  
import pyartm_datasets
from pyartm_datasets import main_cases
from pyartm_datasets import sklearn_dataset
from pyartm_datasets import nips
from pyartm_datasets import twitter_sentiment140
from sklearn.datasets import fetch_20newsgroups
from pyartm.optimizations import balanced
%matplotlib inline

EPS = 2**(-15)
DOCS = 2000
WORDS = 10000
WORDS_IN_DOC = 1000
TOPICS = 100
ARTM_DIR = os.path.dirname(os.path.realpath(pyartm_datasets.__file__))
ARTM_RESOURCES = os.path.join(ARTM_DIR, 'resources')
DATASETS_PATH = os.environ.get('PYARTM_DATASETS_PATH', os.path.join(os.path.expanduser('~'), 'pyartm-datasets'))
NIPS_PATH = os.path.join(DATASETS_PATH, 'NIPS.csv')
TWITTER_SENTIMENT140_PATH = os.path.join( DATASETS_PATH, 'twitter-sentiment140.csv')
WNTM_MATRIX_DIR_PATH = os.path.join(DATASETS_PATH, 'wntm_matrix')
ITERS_COUNT = 401
SAMPLES = 1
DEFAULT_EXP_PATH = 'drive/MyDrive/experiment.pkl'
DEFAUL_EXP_PHI_PATH = 'drive/MyDrive/experimentphi.pkl'


nltk.download('stopwords')


def prepare(
        dataset,
        train_proportion=None,
        min_occurrences = 3,
        token_2_num = None,
        process_log_step = 500
):
    english_stopwords = set(stopwords.words('english'))
    is_token_2_num_provided = token_2_num is not None
    occurrences = None

    # remove stopwords
    if not is_token_2_num_provided:
        token_2_num = dict()
        occurrences = Counter()
        for i, doc in enumerate(dataset.data):
            tokens = gensim.utils.lemmatize(doc)
            for token in set(tokens):
                occurrences[token] += 1
            if i % process_log_step == 0:
                print('Preprocessed: ', i, 'documents from', len(dataset.data))

    row, col, data = [], [], []
    row_test, col_test, data_test = [], [], []
    not_empty_docs_number = 0
    doc_targets = []
    random_gen = random.Random(42)

    for i, (doc, target) in enumerate(
        zip(dataset.data, dataset.target)
    ):
        tokens = gensim.utils.lemmatize(doc)
        cnt = Counter()
        cnt_test = Counter()
        for token in (set(tokens)):
            word = (token.decode("utf-8")).split('/')[0]
            if (
                    not is_token_2_num_provided
                    and word not in english_stopwords
                    and min_occurrences <= occurrences[token]
                    and token not in token_2_num
            ):
                token_2_num[token] = len(token_2_num)
            if token in token_2_num:
                if (
                        train_proportion is None
                        or random_gen.random() < train_proportion
                ):
                    cnt[token_2_num[token]] += 1
                else:
                    cnt_test[token_2_num[token]] += 1

        if len(cnt) > 0 and (train_proportion is None or len(cnt_test) > 0):
            for w, c in iteritems(cnt):
                row.append(not_empty_docs_number)
                col.append(w)
                data.append(c)

            for w, c in iteritems(cnt_test):
                row_test.append(not_empty_docs_number)
                col_test.append(w)
                data_test.append(c)

            not_empty_docs_number += 1
            doc_targets.append(target)

        if i % process_log_step == 0:
            print('Processed: ', i, 'documents from', len(dataset.data))

    num_2_token = {
        v: k
        for k, v in iteritems(token_2_num)
    }

    shape = (not_empty_docs_number, len(token_2_num))
    if train_proportion is None:
        return (
            scipy.sparse.csr_matrix((data, (row, col)), shape=shape),
            token_2_num,
            num_2_token,
            doc_targets
        )
    else:
        return (
            scipy.sparse.csr_matrix((data, (row, col)), shape=shape),
            scipy.sparse.csr_matrix(
                (data_test, (row_test, col_test)), shape=shape
            ),
            token_2_num,
            num_2_token,
            doc_targets
        )
def get_resource_path(name):
    return os.path.join(ARTM_RESOURCES, name)
    
def get_20newsgroups(categories = None, min_occurrences=3, train_proportion=None, subset='all'):
    path = get_resource_path('20newsgroups_subset_{}_{}_{}.pkl'.format(
        subset, min_occurrences, train_proportion))
    
    if os.path.exists(path):
        with open(path, 'rb') as resource_file:
            data = pickle.load(resource_file)
    else:
        data = prepare(
            fetch_20newsgroups(
                subset=subset,
                categories=categories,
                remove=('headers', 'footers', 'quotes'), random_state= 100
            ),
            min_occurrences=min_occurrences,
            train_proportion=train_proportion
        )
        with open(path, 'wb') as resource_file:
            pickle.dump(data, resource_file)
    return data




In [ ]:
def simil_fun_cdist(a, b):
    return ( cdist(a.reshape(1,len(a)), b.reshape(1,len(b)),'cosine'))

def simil_fun_euclid(a, b):
    return (np.sum((a - b) * (a - b)))

def simil_fun_hellinger(a, b):
    return (norm(np.sqrt(a) - np.sqrt(b)) / np.sqrt(2))

def simil_fun_jaccard_5(a, b):
    a_bin = a >= 10**(-5)
    b_bin = b >= 10**(-5)
    return scipy.spatial.distance.jaccard(a, b)

def simil_fun_jaccard_4(a,b):
    a_bin = a >= 10**(-4)
    b_bin = b >= 10**(-4)
    return scipy.spatial.distance.jaccard(a, b)

def simil_fun_jenson_shennon(a, b):
    return scipy.spatial.distance.jensenshannon(a, b)

def count_metrics_for_phi(phi, phi_0, simil_fun, num_topics):
    distance = np.zeros((num_topics,num_topics))
    for i in range(num_topics):
        for j in range(num_topics):
            distance[i][j] = simil_fun(phi[:,i],phi_0[:,j])
    ax0 = np.argmin(distance, axis = 0)
    ax1 = np.argmin(distance, axis = 1)
    uni0 = np.unique(ax0, return_counts = True)
    uni1 = np.unique(ax1, return_counts = True)
    f1 = pairsim(ax0, ax1)
    f2 = (np.sum((uni0[1] > 1))) # 
    f3 = (np.sum((uni1[1] > 1))) #
    f4 = (num_topics - len(uni0[0]))
    f5 = (num_topics - len(uni1[0]))
    return(f1, f2, f3, f4, f5)

def count_metrics_values_for_phi(phi, phi_0, simil_fun, num_topics):
    distance = np.zeros((num_topics,num_topics))
    for i in range(num_topics):
        for j in range(num_topics):
            distance[i][j] = simil_fun(phi[:,i],phi_0[:,j])
    ax0 = np.min(distance, axis = 0)
    return np.sum(ax0)

def check(phi_true, phi_test, num_topics):
    m1 = count_metrics_for_phi(phi_true, phi_test, simil_fun_cdist, num_topics)
    m2 = count_metrics_for_phi(phi_true, phi_test, simil_fun_euclid, num_topics)
    m3 = count_metrics_for_phi(phi_true, phi_test, simil_fun_jenson_shennon, num_topics)
    m4 = count_metrics_for_phi(phi_true, phi_test, simil_fun_hellinger, num_topics)
    print(m1)
    print(m2) 
    print(m3) 
    print(m4)
    return(m1,m2,m3,m4)

def asym_count_metrics_values_for_phi(phi, phi_0, simil_fun, num_topics_phi, num_topics_phi_0):
    distance = np.zeros((num_topics_phi,num_topics_phi_0))
    for i in range(num_topics_phi):
        for j in range(num_topics_phi_0):
            distance[i][j] = simil_fun(phi[:,i],phi_0[:,j])
    ax0 = np.min(distance, axis = 0)
    return np.sum(ax0)
def asym_count_metrics_for_phi(phi, phi_0, simil_fun,num_topics_phi, num_topics_phi_0):
    distance = np.zeros((num_topics_phi,num_topics_phi_0))
    for i in range(num_topics_phi):
        for j in range(num_topics_phi_0):
            distance[i][j] = simil_fun(phi[:,i],phi_0[:,j])
    ax0 = np.argmin(distance, axis = 0)
    ax1 = np.argmin(distance, axis = 1)
    uni0 = np.unique(ax0, return_counts = True)
    uni1 = np.unique(ax1, return_counts = True)
    f1 = pairsim(ax0, ax1)
    f2 = (np.sum((uni0[1] > 1))) # 
    f3 = (np.sum((uni1[1] > 1))) #
    f4 = (num_topics_phi - len(uni0[0]))
    f5 = (num_topics_phi_0 - len(uni1[0]))
    return(f1, f2, f3, f4, f5)
def asym_check(phi_true, phi_test, num_topics_phi,num_topics_phi_0):
    m1 = asym_count_metrics_for_phi(phi_true, phi_test, simil_fun_cdist, num_topics_phi,num_topics_phi_0)
    m2 = asym_count_metrics_for_phi(phi_true, phi_test, simil_fun_euclid, num_topics_phi,num_topics_phi_0)
    m3 = asym_count_metrics_for_phi(phi_true, phi_test, simil_fun_jenson_shennon,num_topics_phi,num_topics_phi_0)
    m4 = asym_count_metrics_for_phi(phi_true, phi_test, simil_fun_hellinger, num_topics_phi,num_topics_phi_0)
    print(m1)
    print(m2) 
    print(m3) 
    print(m4)
    return(m1,m2,m3,m4)

In [ ]:
def perform_experiment(
   train_n_dw_matrix, test_n_dw_matrix, optimizer,
   T, samples, output_path , tau , path_phi_output 
):

    optimizer.iteration_callback = experiments.default_callback(
        train_n_dw_matrix = train_n_dw_matrix,
        test_n_dw_matrix = test_n_dw_matrix,
        top_pmi_sizes = [5, 10, 20, 30],
        top_avg_jaccard_sizes = [10, 50, 100, 200],
        measure_time = False
    )

    for seed in tqdm(range(samples)):
        realseed = seed 
        expphi,exptheta = experiments.default_sample(train_n_dw_matrix, T, realseed, optimizer,tau = tau )
        with open(path_phi_output + str(realseed), 'wb') as resource_file:
            pickle.dump(expphi, resource_file) 
        with open(path_phi_output + 'theta' + str(realseed ), 'wb') as resource_file:
            pickle.dump(exptheta, resource_file)
    optimizer.iteration_callback.save_results(output_path + str(realseed ))
    
    return (expphi, exptheta)

def build_dataset(N_topics = 20, min_docs_in_topic = 30)
    data = np.zeros((min_docs_in_topic*N_topics*(N_topics+1)/2, 30511))
    train_n_dw_matrix = get_20newsgroups()
    c2 = 0
    for i in range(N_topics):
        c1 = 0
        j = 0
        while (j < min_docs_in_topic*(i+1)):
            if(np.array(train_n_dw_matrix)[3][c1] == i):
                data[c2] = train_n_dw_matrix[0][c1].toarray()
                j = j + 1
                c2 = c2 + 1
            c1 = c1 + 1
    data =  sparse.csr_matrix(data)
    with open("generated_data", 'wb') as resource_file:
        pickle.dump(data, resource_file)
    with open("generated_data", 'rb') as resource_file:
        data = pickle.load(resource_file)
    data = data.toarray()
    short_indexes = []
    for i in tqdm(range(30511)):
        if(data[:,i].sum() < 2):
            short_indexes.append(i)
            data = np.delete(data,short_indexes,axis = 1)
    data =  sparse.csr_matrix(data)
    with open("generated_data_short", 'wb') as resource_file:
        pickle.dump(data, resource_file)
    theta_true = np.zeros(((N_topics+1)*N_topics*min_docs_in_topic/2 ,N_topics))
    ntopics = 0
    docs = 0
    i = 0
    while(ntopics < N_topics):
        while (docs < (ntopics+1)*(min_docs_in_topic)):
            theta_true[i][c1] = 1
            docs += 1
            i += 1
        docs = 0
        ntopics += 1
    return data,theta

In [ ]:
ITERS_COUNT = 400
SAMPLES = 5
regularization_list = [regularizers.Trivial()] * ITERS_COUNT
def update(results, metric, i,j):
    for k in range(5):
        results[k][i][j] = metric[k]
    return results
def perform_main_experiment(train_n_dw_matrix, test_n_dw_matrix, optimizer,Tarray, 
                            samples, output_path , tau , path_phi_output )
    data,theta = build_dataset()
    for T in Tarray:
        perform_experiment(data, None,default.Optimizer(regularization_list), T, SAMPLES, 
                           output_path = 'experiment_res_tau05_numtopics{}_'.format(T),
                       tau = 0.5 ,path_phi_output = 'dexperiment_res_tau05_numtopics{}_'.format(T))

        perform_experiment(data, None,default.Optimizer(regularization_list), T, SAMPLES, 
                           output_path = 'experiment_res_tauNone_numtopics{}_'.format(T),
                           tau = None ,path_phi_output = 'experiment_res_tauNone_numtopics{}_'.format(T))
    results = np.zeros((5,4,len(T)))
    i = 0
    for T in Tarray:
        with open("experiment_res_tauNone_numtopics{}_theta0".format(T), 'rb') as resource_file:
              experimental_theta = pickle.load(resource_file)
        m1,m2,m3,m4 = asym_check(theta, experimental_theta, 20, T)
        results = update(results,m1,1,i)
        results = update(results,m2,2,i)
        results = update(results,m3,3,i)
        results = update(results,m4,4,i)
    return results